In [1]:
import torch
from torch_geometric.data import Data
from tqdm import tqdm, trange

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [2]:
data

Data(edge_index=[2, 4], x=[3, 1])

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [5]:
from torch_geometric.datasets import QM9

In [13]:
from torch_geometric.data import DataLoader
import numpy as np
from tqdm import tqdm
from torch.utils.data.sampler import SubsetRandomSampler


dataset = QM9(root='/tmp/QM9')

validation_split = .2
shuffle_dataset = True
random_seed= 42
batch_size = 16

# Creating data indices for training and validation splits
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [14]:
from neuralnet.delta_net import *
from neuralnet.egnn_sparse import *
from neuralnet.net_utils import *
from neuralnet.h5_net import *

In [15]:
#h5n = H5Net(n_outputs=1, aggr='add')

In [16]:
#h5n.to(device='cuda')

In [23]:
model_path = 'models/'

In [25]:
h5n = H5Net(n_outputs=1, aggr='add')
h5n.load_state_dict(torch.load(model_path + 'h5n.pth'))
torch.cuda.empty_cache()
h5n.to('cuda')

H5Net(
  (embedding): Embedding(5, 24)
  (emb_ffl): Linear(in_features=24, out_features=24, bias=True)
  (kernels): ModuleList(
    (0): EGNN_sparse(
      (dropout): Dropout(p=0.1, inplace=False)
      (edge_norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (edge_norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (edge_mlp): Sequential(
        (0): Linear(in_features=57, out_features=114, bias=True)
        (1): Dropout(p=0.1, inplace=False)
        (2): SiLU()
        (3): Linear(in_features=114, out_features=64, bias=True)
        (4): SiLU()
      )
      (node_norm1): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
      (node_norm2): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
      (node_mlp): Sequential(
        (0): Linear(in_features=88, out_features=48, bias=True)
        (1): Dropout(p=0.1, inplace=False)
        (2): SiLU()
        (3): Linear(in_features=48, out_features=24, bias=True)
      )
    )
    (1): EGNN_sparse(
   

In [19]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(h5n.parameters(), lr=0.000001)
label_dict = {1: 0, 6: 1, 7: 2, 8: 3, 9: 4}

n_epochs = 100

for e in range(n_epochs):
    training_loss = 0
    t_bar = tqdm(enumerate(train_loader), desc='TRAINING')
    for b,  batch in t_bar:
        batch.z = torch.Tensor([label_dict[int(x)] for x in batch.z]).type(torch.int64)
        batch.to('cuda')
        pred = h5n(batch)
        target = batch.y[:, 13][:, None] / 120  # very crude normalization! (should substract upper & lower limit etc)
        loss = criterion(-1 * pred, target)
        loss.backward()
        optimizer.step()
        training_loss = (training_loss * b + loss) / (b + 1)
        if b % 100 == 0:
            t_bar.set_description('TRAINING: mean epoch loss = '+str(round(float(training_loss), 6)))
    
    with torch.no_grad():
        validation_loss = 0
        t_bar = tqdm(enumerate(validation_loader), desc='VALIDATION')
        for b,  batch in t_bar:
            batch.z = torch.Tensor([label_dict[int(x)] for x in batch.z]).type(torch.int64)
            batch.to('cuda')
            pred = h5n(batch)
            target = batch.y[:, 13][:, None] / 120
            loss = criterion(-1 * pred, target)
            validation_loss = (validation_loss * b + loss) / (b + 1)
            if b % 100 == 0:
                t_bar.set_description('VALIDATION: mean epoch loss = '+str(round(float(validation_loss), 6)))

TRAINING: mean epoch loss = 0.008161: : 6542it [01:21, 80.30it/s]
VALIDATION: mean epoch loss = 0.003744: : 1636it [00:09, 171.12it/s]
TRAINING: mean epoch loss = 0.002681: : 6542it [01:10, 92.90it/s]
VALIDATION: mean epoch loss = 0.00227: : 1636it [00:07, 228.47it/s] 
TRAINING: mean epoch loss = 0.002101: : 6542it [01:09, 94.49it/s]
VALIDATION: mean epoch loss = 0.002026: : 1636it [00:07, 227.90it/s]
TRAINING: mean epoch loss = 0.002077: : 6542it [01:09, 94.20it/s]
VALIDATION: mean epoch loss = 0.002061: : 1636it [00:07, 219.70it/s]
TRAINING: mean epoch loss = 0.002307: : 6542it [01:08, 95.91it/s]
VALIDATION: mean epoch loss = 0.002689: : 1636it [00:07, 231.22it/s]
TRAINING: mean epoch loss = 0.002283: : 6542it [01:07, 96.26it/s]
VALIDATION: mean epoch loss = 0.002047: : 1636it [00:07, 232.53it/s]
TRAINING: mean epoch loss = 0.002217: : 6542it [01:08, 96.05it/s]
VALIDATION: mean epoch loss = 0.00258: : 1636it [00:07, 232.07it/s] 
TRAINING: mean epoch loss = 0.002104: : 6542it [01:08, 

VALIDATION: mean epoch loss = 0.001697: : 1636it [00:07, 220.17it/s]
TRAINING: mean epoch loss = 0.001685: : 6542it [01:07, 96.58it/s]
VALIDATION: mean epoch loss = 0.001695: : 1636it [00:07, 232.00it/s]
TRAINING: mean epoch loss = 0.00169: : 6542it [01:08, 96.17it/s] 
VALIDATION: mean epoch loss = 0.001716: : 1636it [00:07, 230.96it/s]
TRAINING: mean epoch loss = 0.001682: : 6542it [01:08, 96.05it/s]
VALIDATION: mean epoch loss = 0.001694: : 1636it [00:07, 231.54it/s]
TRAINING: mean epoch loss = 0.001682: : 6542it [01:08, 96.08it/s]
VALIDATION: mean epoch loss = 0.001707: : 1636it [00:07, 231.77it/s]
TRAINING: mean epoch loss = 0.001682: : 6542it [01:07, 96.85it/s]
VALIDATION: mean epoch loss = 0.001694: : 1636it [00:07, 232.10it/s]
TRAINING: mean epoch loss = 0.001677: : 6542it [01:07, 97.10it/s]
VALIDATION: mean epoch loss = 0.001704: : 1636it [00:07, 221.06it/s]
TRAINING: mean epoch loss = 0.001675: : 6542it [01:07, 97.33it/s]
VALIDATION: mean epoch loss = 0.001711: : 1636it [00:07

In [24]:
torch.save(h5n.state_dict(), model_path+'h5n.pth')